## Folder Path

In [1]:
input_folder_path = "../../Input/csv_files/"

## Libraries Import

In [2]:
from neo4j import GraphDatabase
import pandas as pd
import os

## Data Loading

In [3]:
df = pd.read_csv(os.path.abspath(os.path.join(input_folder_path, "sample_cast_1.csv")))

## Neo4j Config

In [4]:
# Load config from file
def load_neo4j_config(filepath):
    config = {}
    with open(filepath, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
    return config

In [5]:
config = load_neo4j_config("../Encryption/Neo4j-1cf46bb6-Created-2025-04-10.txt")

In [6]:
uri = config.get("NEO4J_URI")
username = config.get("NEO4J_USERNAME")
password = config.get("NEO4J_PASSWORD")

## Driver Initialisation

In [7]:
# Create driver
driver = GraphDatabase.driver(uri, auth=(username, password))

## Test Connection

In [8]:
# Example query to test
def test_connection():
    with driver.session() as session:
        result = session.run("RETURN 'Connection successful!' AS message")
        print(result.single()["message"])

In [9]:
test_connection()

Connection successful!


## Data Loading

In [10]:
def load_data(tx, personId, name, birthYear):
    query = """
        MERGE (p: Person {personId: $personId})
        SET p.name = $name, p.birthYear = $birthYear
    """
    
    tx.run(query, personId=personId, name=name, birthYear=birthYear)

In [11]:
with driver.session() as session:
    for index, row in df.iterrows():
        session.execute_write(load_data, row['personId'], row['name'], row['birthYear'])
        
print("CSV file successfully loaded into Neo4j.")

CSV file successfully loaded into Neo4j.


## View Data

In [12]:
def view_data(tx):
    query = """
        MATCH (p:Person)
        RETURN p.personId AS personId, p.name AS name, p.birthYear AS birthYear
    """
    result = tx.run(query)
    
    return [record.data() for record in result]

In [13]:
with driver.session() as session:
    people = session.execute_read(view_data)
    
    for person in people:
        print(person)

{'personId': 23945, 'name': 'Gerard Pires', 'birthYear': 1942}
{'personId': 553509, 'name': 'Helen Reddy', 'birthYear': 1941}
{'personId': 113934, 'name': 'Susan Flannery', 'birthYear': 1939}


In [14]:
driver.close()